# DMN-Virtual Env Replication
In this notebook, we re-run our initial analyses from our first submission to ensure that our data continues to be accessible. Unfortunately, we discovered that many of the packages that we initially relied on depreciated. To use this code, you have two options: you can access a Docker image created by Alejandro de la Vega, which is listed on his repo <a href="https://github.com/adelavega/neurosynth-mfc">here</a>. Alternatively, you can create a virtual environment with all of your dependencies set to the correct version. The latter option worked best for us. 

In [1]:
%matplotlib inline
from neurosynth.base.dataset import Dataset
dataset = Dataset.load('neurosynth-lfc/data/data/neurosynth_60_0.6.pkl')

In [2]:
cognitive_topics = ['memory','categorization','switching','inhibition','priming',
                    'social','fear','emotion','learning','reward',  
                    'decision-making','imagery','spatial','attention','WM',
                    'awareness','language','math','semantics','reading']

In [3]:
disease_topics = ['smoking','eating-disorder','depression','schizopherenia','adhd','autism','Alzheimer-Parkinson','ptsd']

# The DMN

In [4]:
from sklearn.naive_bayes import GaussianNB
from classification import RegionalClassifier
from sklearn.metrics import roc_auc_score

In [5]:
clf = RegionalClassifier(dataset, 'dmn-parcellation/masks/DMN.nii.gz', GaussianNB())
clf.classify(scoring=roc_auc_score)

Classifying...
[##########] 100%


In [6]:
clf.class_score

array([0.63953177, 0.59314009, 0.59947901, 0.56496712])

In [7]:
import pandas as pd

nicknames = pd.read_csv('neurosynth-lfc/data/v4-topics-60.txt', delimiter='\t')
nicknames['topic_name'] = nicknames.apply(lambda row: '_'.join([str(row.topic_number)] + row.top_words.split(' ')[0:3]), axis=1)
nicknames = nicknames.sort_values('topic_name')

In [8]:
formatted_importances = clf.get_formatted_importances(feature_names=nicknames.nickname)

In [9]:
#formatted_importances.to_csv('dmn_formatted_importances.csv')

### DMN Permutations 

In [9]:
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [10]:
%load_ext autoreload
%autoreload 1
%aimport classification

In [11]:
# These are the names of each region in the DMN
names_70 = ['1','2','3','4']

In [12]:
from classification import permute_log_odds
lor_z = classification.permute_log_odds(clf, 1000, feature_names=nicknames.nickname, region_names = names_70)

[##########] 100%


In [13]:
cog_ps = lor_z[lor_z.nickname.isin(cognitive_topics)]
from statsmodels.sandbox.stats.multicomp import multipletests

reject, p_corr, a, a1 = multipletests(cog_ps.p, alpha=0.01, method='fdr_tsbky')

cog_ps['p_corr_01'] = p_corr # Adjusted p-value
cog_ps['reject_01'] = (cog_ps.p_corr_01<0.05) & (cog_ps.lor_z > 0) # Was the null hypothesis rejected?

/anaconda3/envs/neurosynth_env/lib/python2.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/envs/neurosynth_env/lib/python2.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


### Obtaining the LOR CI's

In [14]:
lor_ci = classification.bootstrap_log_odds(clf, 100, feature_names=nicknames.nickname, 
                                           region_names = names_70, n_jobs=3)

[##########] 100%


classification.py:233: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  return overall_boot.groupby(['region', 'topic_name'])['fi'].agg({'mean' : np.mean, 'low_ci' : percentile(2.5), 'hi_ci' : percentile(97.5)}).reset_index()


In [16]:
lor_ci.rename({'topic_name': 'feature'}, axis=1, inplace=True)
lor_ci['ci_range']=lor_ci['hi_ci']-lor_ci['low_ci']
lor_ci.drop(['hi_ci','low_ci','mean'], axis=1, inplace=True)
lor_ci['region']=lor_ci['region'].astype(int, inplace=True)
merge = pd.merge(formatted_importances, lor_ci, how='left', left_on=['region','feature'], right_on=['region','feature'])

In [18]:
#merge.to_csv('final-output/dmn_data.csv')

# The MPFC

In [63]:
names_70=[1,2]

clf = RegionalClassifier(dataset, 'dmn-parcellation/images/MPFC/cluster_labels_k2.nii.gz', GaussianNB())
clf.classify(scoring=roc_auc_score)
formatted_importances = clf.get_formatted_importances(feature_names=nicknames.nickname)

Classifying...
[##########] 100%


In [64]:
clf.class_score

array([0.62871038, 0.62320936])

### Obtaining the LOR CI's

In [65]:
lor_ci = classification.bootstrap_log_odds(clf, 100, feature_names=nicknames.nickname, 
                                           region_names = names_70, n_jobs=3)

[##########] 100%


In [66]:
lor_ci.rename({'topic_name': 'feature'}, axis=1, inplace=True)
lor_ci['ci_range']=lor_ci['hi_ci']-lor_ci['low_ci']
lor_ci.drop(['hi_ci','low_ci','mean'], axis=1, inplace=True)
merge = pd.merge(formatted_importances, lor_ci, how='left', left_on=['region','feature'], right_on=['region','feature'])

In [68]:
merge.to_csv('final-output/mpc-data.csv')

# The PCC

In [69]:
names_70=[1,2,3]

clf = RegionalClassifier(dataset, 'dmn-parcellation/images/PCC/cluster_labels_k3.nii.gz', GaussianNB())
clf.classify(scoring=roc_auc_score)
formatted_importances = clf.get_formatted_importances(feature_names=nicknames.nickname)

Classifying...
[##########] 100%


In [70]:
clf.class_score

array([0.57195308, 0.59751122, 0.57454637])

In [71]:
lor_ci = classification.bootstrap_log_odds(clf, 100, feature_names=nicknames.nickname, 
                                           region_names = names_70, n_jobs=3)

[##########] 100%


In [72]:
lor_ci.rename({'topic_name': 'feature'}, axis=1, inplace=True)
lor_ci['ci_range']=lor_ci['hi_ci']-lor_ci['low_ci']
lor_ci.drop(['hi_ci','low_ci','mean'], axis=1, inplace=True)
merge = pd.merge(formatted_importances, lor_ci, how='left', left_on=['region','feature'], right_on=['region','feature'])

In [74]:
#merge.head()
#merge.to_csv('final-output/pcc-data.csv')

# The lTPJ

In [75]:
names_70=[1,2,3]

clf = RegionalClassifier(dataset, 'dmn-parcellation/images/lTPJ/cluster_labels_k3.nii.gz', GaussianNB())
clf.classify(scoring=roc_auc_score)
formatted_importances = clf.get_formatted_importances(feature_names=nicknames.nickname)

Classifying...
[##########] 100%


In [76]:
clf.class_score

array([0.59309422, 0.57119875, 0.59140037])

In [77]:
lor_ci = classification.bootstrap_log_odds(clf, 100, feature_names=nicknames.nickname, 
                                           region_names = names_70, n_jobs=3)

[##########] 100%


In [78]:
lor_ci.rename({'topic_name': 'feature'}, axis=1, inplace=True)
lor_ci['ci_range']=lor_ci['hi_ci']-lor_ci['low_ci']
lor_ci.drop(['hi_ci','low_ci','mean'], axis=1, inplace=True)
merge = pd.merge(formatted_importances, lor_ci, how='left', left_on=['region','feature'], right_on=['region','feature'])

In [80]:
#merge.to_csv('final-output/ltpj-data.csv')

# The rTPJ

In [81]:
names_70=[1,2,3]

clf = RegionalClassifier(dataset, 'dmn-parcellation/images/rTPJ/cluster_labels_k3.nii.gz', GaussianNB())
clf.classify(scoring=roc_auc_score)
formatted_importances = clf.get_formatted_importances(feature_names=nicknames.nickname)

Classifying...
[##########] 100%


In [82]:
clf.class_score

array([0.55625326, 0.56891932, 0.56224357])

In [83]:
lor_ci = classification.bootstrap_log_odds(clf, 100, feature_names=nicknames.nickname, 
                                           region_names = names_70, n_jobs=3)

[##########] 100%


In [84]:
lor_ci.rename({'topic_name': 'feature'}, axis=1, inplace=True)
lor_ci['ci_range']=lor_ci['hi_ci']-lor_ci['low_ci']
lor_ci.drop(['hi_ci','low_ci','mean'], axis=1, inplace=True)
merge = pd.merge(formatted_importances, lor_ci, how='left', left_on=['region','feature'], right_on=['region','feature'])

In [86]:
#merge.to_csv('final-output/rtpj-data.csv')